#**CNN Image processing with NOISY data...
Dr. Munir Ahmad, Lahore, PKISTAN
munirahm@gmail.com**

# **We can import the needed libraries first....**

In [ ]:
from keras.datasets import cifar10
from skimage.util import random_noise
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm


# **Load CIFAR10 dataset and normlize traning and test data....**

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
nX_train = X_train
nX_test = X_test

# **Add noise to the dataset with variance of 0.01**

In [ ]:
for img in range(X_train.shape[0]):
  nX_train[img, :, :, :] = random_noise(X_train[img, :, :, :], mode='gaussian', seed=None, clip=True, var=0.01)
for img in range(X_test.shape[0]):
  nX_test[img, :, :, :] = random_noise(X_test[img, :, :, :], mode='gaussian', seed=None, clip=True, var=0.01)
  

# **Devise a model based on Vgg...**

In [ ]:
drp = 0.2
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(Dropout(drp))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(drp))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(drp))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(Dropout(drp))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(drp))
model.add(BatchNormalization())

model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(drp))
model.add(BatchNormalization())

model.add(Dense(10, 'softmax'))

epochs = 25
optimizer = 'adam'

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
print(model.summary())

#**Modelrun with nioseless data...**

In [ ]:
np.random.seed(21)
history1 = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

#**Noiseless results...**

In [ ]:
# list all data in history
print(history1.history.keys())
# summarize history for accuracy
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model sloss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
np.random.seed(21)
history = model.fit(nX_train, y_train, validation_data=(nX_test, y_test), epochs=epochs, batch_size=64)

# **Display accuracy and loss curves...**

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()